Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [59]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [60]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


## Lab3

In [61]:
median = np.median(DIST_MATRIX.reshape(1, -1))
ic(median)
# DIST_MATRIX[DIST_MATRIX > median] = np.inf 
G = nx.Graph()
for c1, c2 in combinations(CITIES.itertuples(), 2):
    G.add_node(c1.Index, name=c1.name)
    G.add_node(c2.Index, name=c2.name)
    if DIST_MATRIX[c1.Index, c2.Index] <= median: #simple tweaking: if the distance btw two cities > median, then make the edge not-connected
        G.add_edge(c1.Index, c2.Index, weight=DIST_MATRIX[c1.Index, c2.Index])
nx.is_connected(G)

ic| median: np.float64(367.0694334013403)


True

In [62]:
ic(list(G.nodes(data=True))[:10])  
ic(list(G.edges(data=True))[:10]) 

start_city = CITIES[CITIES['name'] == 'Turin'].index[0]
end_city = CITIES[CITIES['name'] == 'Bari'].index[0]

if start_city in G and end_city in G:

    shortest_path = nx.shortest_path(G, source=start_city, target=end_city, weight='weight')
    path_distance = nx.shortest_path_length(G, source=start_city, target=end_city, weight='weight')
    ic("Shortest path:", [CITIES.loc[node, 'name'] for node in shortest_path])
    ic("Path distance:", path_distance)
else:
    print("Selected cities are not in the connected Graph")

ic| list(G.nodes(data=True))[:10]: [(0, {'name': 'Ancona'}),
                                    (1, {'name': 'Andria'}),
                                    (2, {'name': 'Bari'}),
                                    (3, {'name': 'Bergamo'}),
                                    (4, {'name': 'Bologna'}),
                                    (5, {'name': 'Bolzano'}),
                                    (6, {'name': 'Brescia'}),
                                    (7, {'name': 'Cagliari'}),
                                    (8, {'name': 'Catania'}),
                                    (9, {'name': 'Ferrara'})]
ic| list(G.edges(data=True))[:10]: [(0, 1, {'weight': np.float64(349.30401144305745)}),
                                    (0, 4, {'weight': np.float64(199.89962990855176)}),
                                    (0, 5, {'weight': np.float64(364.0447808729763)}),
                                    (0, 6, {'weight': np.float64(338.8096401206489)}),
                                  